In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out

class EEGNetTransformer(nn.Module):
    def __init__(self, in_channels, num_classes, num_layers, hidden_dim):
        super(EEGNetTransformer, self).__init__()
        self.input_conv = nn.Conv1d(in_channels, 64, kernel_size=3, padding=1)
        self.conv_block1 = ConvBlock(64, 128)
        self.conv_block2 = ConvBlock(128, 256)
        self.transformer = nn.Transformer(d_model=256, nhead=8, num_encoder_layers=num_layers, dim_feedforward=hidden_dim)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, src):
        out = self.input_conv(src)
        out = self.conv_block1(out)
        out = self.conv_block2(out)
        out = out.permute(0, 2, 1)  # 调整维度顺序
        out = self.transformer(out, out)  # 将out作为源数据和目标数据传递给Transformer
        out = out.permute(0, 2, 1)  # 还原维度顺序
        out = torch.mean(out, dim=2)  # Global average pooling
        out = self.fc(out)
        return out

# 创建模型实例
model = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)



# 创建一个示例网络
in_channels = 1  # 输入信号的通道数
num_classes = 2  # 分类的类别数
num_layers = 4  # Transformer中的层数
hidden_dim = 512  # 前馈神经网络的隐藏层维度
eegnet_transformer = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)

# 打印网络结构
print(eegnet_transformer)


NameError: name 'in_channels' is not defined

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, f1_score
import numpy as np

# 读取数据集
class EEGDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = self.data.iloc[idx, :-1].values.astype(float)
        label = self.data.iloc[idx, -1]
        # 调整数据维度顺序以匹配Conv1d的输入要求
        features = features.reshape(1, -1)  # 从 [sequence_length, in_channels] 调整为 [in_channels, sequence_length]
        return torch.Tensor(features), label




# 定义超参数
in_channels = 1
num_classes = 2
num_layers = 4
hidden_dim = 512
batch_size = 16
num_epochs = 80
learning_rate = 0.0001

# 创建数据加载器
dataset = EEGDataset('database.csv')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 创建模型实例
model = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
total_step = len(dataloader)
for epoch in range(num_epochs):
    model.train()  # 确保模型处于训练模式
    progress_bar = tqdm(enumerate(dataloader), total=total_step, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
    
    # 初始化用于计算评估指标的列表
    all_predictions = []
    all_labels = []
    
    for i, (data, labels) in progress_bar:
        # 前向传播
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)

        # 反向传播和优化

        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'Loss': loss.item()})
    
    # 计算评估指标
    accuracy = accuracy_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    
    # 打印评估结果
    print(f'\nEpoch {epoch+1}/{num_epochs}, Accuracy: {accuracy*100:.2f}%, Recall: {recall:.2f}, F1 Score: {f1:.2f}\n')

Epoch 1/80: 100%|██████████| 13/13 [00:01<00:00,  9.66batch/s, Loss=0.636]



Epoch 1/80, Accuracy: 54.50%, Recall: 0.55, F1 Score: 0.53



Epoch 2/80: 100%|██████████| 13/13 [00:01<00:00, 10.01batch/s, Loss=0.853]



Epoch 2/80, Accuracy: 54.00%, Recall: 0.54, F1 Score: 0.53



Epoch 3/80: 100%|██████████| 13/13 [00:01<00:00, 10.33batch/s, Loss=0.0564]


...


Epoch 35/80, Accuracy: 99.00%, Recall: 0.99, F1 Score: 0.99



Epoch 36/80: 100%|██████████| 13/13 [00:01<00:00, 10.63batch/s, Loss=0.00898]



Epoch 36/80, Accuracy: 99.50%, Recall: 0.99, F1 Score: 1.00



Epoch 37/80:  54%|█████▍    | 7/13 [00:00<00:00, 10.20batch/s, Loss=0.00484]


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, f1_score
import numpy as np

# 读取数据集
class EEGDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = self.data.iloc[idx, :-1].values.astype(float)
        label = self.data.iloc[idx, -1]
        # 调整数据维度顺序以匹配Conv1d的输入要求
        
        features = features.reshape(1, -1)  # 从 [sequence_length, in_channels] 调整为 [in_channels, sequence_length]
        return torch.Tensor(features), label




# 定义超参数
in_channels = 1
num_classes = 2
num_layers = 4
hidden_dim = 512
batch_size = 256
num_epochs = 20
learning_rate = 0.0001
save_interval = 5  # 每隔多少个epoch保存一次模型

# 指定默认设备为GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 创建数据加载器
dataset = EEGDataset('chbmit_preprocessed_data.csv')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 创建模型实例并移动到GPU上
model = EEGNetTransformer(in_channels, num_classes, num_layers, hidden_dim)
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
total_step = len(dataloader)
losses = []
accuracies = []
recalls = []
f1_scores = []

for epoch in range(num_epochs):
    model.train()  # 确保模型处于训练模式
    progress_bar = tqdm(enumerate(dataloader), total=total_step, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
    
    # 初始化用于计算评估指标的列表
    all_predictions = []
    all_labels = []
  
    for i, (data, labels) in progress_bar:
        # 将数据移动到GPU上
        data = data.to(device)
        labels = labels.to(device)
        
        # 前向传播
        optimizer.zero_grad()
        outputs = model(data)
        labels = labels.long()  # 将标签转换为Long类型
        loss = criterion(outputs, labels)

        # 反向传播和优化
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'Loss': loss.item()})
    
    # 计算评估指标
    accuracy = accuracy_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    
    # 添加损失和评估指标到列表中
    losses.append(loss.item())
    accuracies.append(accuracy)
    recalls.append(recall)
    f1_scores.append(f1)
    
    # 打印评估结果
    print(f'\nEpoch {epoch+1}/{num_epochs}, Accuracy: {accuracy*100:.2f}%, Recall: {recall:.2f}, F1 Score: {f1:.2f}\n')
    
    # 每隔save_interval个epoch保存模型
    if (epoch+1) % save_interval == 0:
        torch.save(model.state_dict(), f'model_epoch{epoch+1}.pth')

# 保存最后一个epoch的模型
torch.save(model.state_dict(), f'model_epoch{num_epochs}.pth')

# 绘制损失曲线
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# 绘制评估指标曲线
plt.plot(accuracies, label='Accuracy')
plt.plot(recalls, label='Recall')
plt.plot(f1_scores, label='F1 Score')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Evaluation Metrics')
plt.legend()
plt.show()



Epoch 1/20: 100%|██████████| 8192/8192 [16:31<00:00,  8.26batch/s, Loss=0.444]



Epoch 1/20, Accuracy: 74.79%, Recall: 0.75, F1 Score: 0.75



Epoch 2/20: 100%|██████████| 8192/8192 [16:30<00:00,  8.27batch/s, Loss=0.434]  



Epoch 2/20, Accuracy: 78.60%, Recall: 0.79, F1 Score: 0.79



Epoch 3/20: 100%|██████████| 8192/8192 [16:41<00:00,  8.18batch/s, Loss=0.393]  


...

Epoch 12/20: 100%|██████████| 8192/8192 [16:40<00:00,  8.19batch/s, Loss=0.307]



Epoch 12/20, Accuracy: 83.06%, Recall: 0.83, F1 Score: 0.83



Epoch 13/20: 100%|██████████| 8192/8192 [16:37<00:00,  8.21batch/s, Loss=0.327]



Epoch 13/20, Accuracy: 83.20%, Recall: 0.83, F1 Score: 0.83



Epoch 14/20:  25%|██▌       | 2062/8192 [04:10<12:30,  8.17batch/s, Loss=0.352]